In [1]:
import numpy as np

3.1 Написать функцию формирования порождающей и проверочной
матриц кода Хэмминга (𝟐
𝒓 − 𝟏, 𝟐
𝒓 − 𝒓 − 𝟏, 𝟑) на основе параметра 𝒓,
а также таблицы синдромов для всех однократных ошибок.


In [2]:
def generate_hamming_code(r, n):

    # Формирование проверочной матрицы H
    H = [[0 for _ in range(r)] for _ in range(n)]
    ii = 0
    for i in range(n):
        x = bin(n - i)[2:] #0b100010101
        if x.count('1') != 1:
            if len(x) < r:
                x = '0' * (r - len(x)) + x
            for j in range(r):
                H[ii][j] = int(x[j])
            ii += 1
    for i in range(r):
        H[ii + i][i] = 1
    H = np.array(H)

    # Формирование порождающей матрицы G
    G = []
    for i in range(ii):
        a = [0 for _ in range(ii)]
        a[i] = 1
        G.append(a)
    for i in range(ii):
        for j in range(r):
            G[i].append(H[i][j])
    G = np.array(G)
    return G, H

In [4]:
def generate_error_table(r, n, H, count_errors):
    error_table = {}
    for i in range(n):
        error = np.zeros(n, dtype=int)
        error[i] = 1
        syndrome = np.dot(error, H) % 2
        error_table[tuple(error)] = tuple(syndrome)

    return error_table

3.2. Провести исследование кода Хэмминга для одно-, двух- и
трёхкратных ошибок для 𝒓 = 𝟐, 𝟑, 𝟒.


In [5]:
def kill_errors(G, H, count_errors, error_table):
    print("Внесение", count_errors, "ошибок в слово")
    word = np.random.randint(0, 2, len(G))
    print('word = ', word)
    code_word = np.dot(word, G) % 2
    print('code_word = ', code_word)
    error_position = np.random.choice(len(G[0] - 1) , count_errors, replace=False)
    print('error_position =', error_position)
    code_word_with_error = np.copy(code_word)
    for i in error_position:
        code_word_with_error[i] = (code_word_with_error[i] + 1) % 2
    print("code_word_with_error = ", code_word_with_error)

    syndrome = np.dot(code_word_with_error, H) % 2
    print("syndrome = ", syndrome)
    possibleErrorPosition = []
    for error, s in error_table.items():
        if tuple(syndrome) == s:
            mass = list(error)
            pos = []
            for i in range(len(mass)):
                if mass[i] == 1:
                    pos.append(i)
            possibleErrorPosition.append(pos)
    # исправление ошибки
    flag = True
    final_word = np.copy(code_word_with_error)
    for i in possibleErrorPosition:
        copy_word = np.copy(code_word_with_error)
        for x in i:
            copy_word[x] = (copy_word[x] + 1) % 2
        # Проверка правильности полученного кодового слова
        flag_error_copy_word = False
        for i in range(len(copy_word)):
            if copy_word[i] != code_word[i]:
                flag_error_copy_word = True
                break
        if not flag_error_copy_word:
            flag = False
            final_word = copy_word
            break
    if flag:
        print("Ошибка не была обнаружена и/или не была успешно исправлена")
        print('code_word = ', code_word)
        print('code_word_without_error = ', final_word)
    else:
        print("Ошибка успешно обнаружена и исправлена")
        print('code_word = ', code_word)
        print('code_word_without_error = ', final_word, '\n')

In [20]:
def errors_stydy(count_errors, r, n, H, G):
    error_table = generate_error_table(r, n, H, count_errors)
    print("\n",'error_table')
    for error, syndrom in error_table.items():
        print(error, '->', syndrom)

    kill_errors(G, H, count_errors, error_table)

In [21]:
def study_for_r(r, n, type_code):
    print("===== Исследование для r =", r, "=====")
    if type_code == 0:
        G, H = generate_hamming_code(r, n)
        print('G = \n', G)
        print('H = \n', H)
        count_errors = 1
        errors_stydy(count_errors, r, n, H, G)

        count_errors = 2
        errors_stydy(count_errors, r, n, H, G)

        count_errors = 3
        errors_stydy(count_errors, r, n, H, G)
    else:
        G, H = generate_extended_hamming_code(r, n)
        print('G = \n', G)
        print('H = \n', H)

        count_errors = 1
        errors_stydy(count_errors, r, n, H, G)

        count_errors = 2
        errors_stydy(count_errors, r, n, H, G)

        count_errors = 3
        errors_stydy(count_errors, r, n, H, G)

        count_errors = 4
        errors_stydy(count_errors, r, n, H, G)

In [22]:
study_for_r(2, 2 ** 2 - 1, 0)

===== Исследование для r = 2 =====
G = 
 [[1 1 1]]
H = 
 [[1 1]
 [1 0]
 [0 1]]

 error_table
(1, 0, 0) -> (1, 1)
(0, 1, 0) -> (1, 0)
(0, 0, 1) -> (0, 1)
Внесение 1 ошибок в слово
word =  [1]
code_word =  [1 1 1]
error_position = [2]
code_word_with_error =  [1 1 0]
syndrome =  [0 1]
Ошибка успешно обнаружена и исправлена
code_word =  [1 1 1]
code_word_without_error =  [1 1 1] 


 error_table
(1, 0, 0) -> (1, 1)
(0, 1, 0) -> (1, 0)
(0, 0, 1) -> (0, 1)
Внесение 2 ошибок в слово
word =  [1]
code_word =  [1 1 1]
error_position = [0 1]
code_word_with_error =  [0 0 1]
syndrome =  [0 1]
Ошибка не была обнаружена и/или не была успешно исправлена
code_word =  [1 1 1]
code_word_without_error =  [0 0 1]

 error_table
(1, 0, 0) -> (1, 1)
(0, 1, 0) -> (1, 0)
(0, 0, 1) -> (0, 1)
Внесение 3 ошибок в слово
word =  [0]
code_word =  [0 0 0]
error_position = [0 2 1]
code_word_with_error =  [1 1 1]
syndrome =  [0 0]
Ошибка не была обнаружена и/или не была успешно исправлена
code_word =  [0 0 0]
code_word_w

In [23]:
study_for_r(3, 2 ** 3 - 1, 0)

===== Исследование для r = 3 =====
G = 
 [[1 0 0 0 1 1 1]
 [0 1 0 0 1 1 0]
 [0 0 1 0 1 0 1]
 [0 0 0 1 0 1 1]]
H = 
 [[1 1 1]
 [1 1 0]
 [1 0 1]
 [0 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]

 error_table
(1, 0, 0, 0, 0, 0, 0) -> (1, 1, 1)
(0, 1, 0, 0, 0, 0, 0) -> (1, 1, 0)
(0, 0, 1, 0, 0, 0, 0) -> (1, 0, 1)
(0, 0, 0, 1, 0, 0, 0) -> (0, 1, 1)
(0, 0, 0, 0, 1, 0, 0) -> (1, 0, 0)
(0, 0, 0, 0, 0, 1, 0) -> (0, 1, 0)
(0, 0, 0, 0, 0, 0, 1) -> (0, 0, 1)
Внесение 1 ошибок в слово
word =  [1 0 0 1]
code_word =  [1 0 0 1 1 0 0]
error_position = [3]
code_word_with_error =  [1 0 0 0 1 0 0]
syndrome =  [0 1 1]
Ошибка успешно обнаружена и исправлена
code_word =  [1 0 0 1 1 0 0]
code_word_without_error =  [1 0 0 1 1 0 0] 


 error_table
(1, 0, 0, 0, 0, 0, 0) -> (1, 1, 1)
(0, 1, 0, 0, 0, 0, 0) -> (1, 1, 0)
(0, 0, 1, 0, 0, 0, 0) -> (1, 0, 1)
(0, 0, 0, 1, 0, 0, 0) -> (0, 1, 1)
(0, 0, 0, 0, 1, 0, 0) -> (1, 0, 0)
(0, 0, 0, 0, 0, 1, 0) -> (0, 1, 0)
(0, 0, 0, 0, 0, 0, 1) -> (0, 0, 1)
Внесение 2 ошибок в слово
word =  [

In [24]:
study_for_r(4, 2 ** 4 - 1, 0)

===== Исследование для r = 4 =====
G = 
 [[1 0 0 0 0 0 0 0 0 0 0 1 1 1 1]
 [0 1 0 0 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 1 1 0 1]
 [0 0 0 1 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 1]]
H = 
 [[1 1 1 1]
 [1 1 1 0]
 [1 1 0 1]
 [1 1 0 0]
 [1 0 1 1]
 [1 0 1 0]
 [1 0 0 1]
 [0 1 1 1]
 [0 1 1 0]
 [0 1 0 1]
 [0 0 1 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]

 error_table
(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) -> (1, 1, 1, 1)
(0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) -> (1, 1, 1, 0)
(0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) -> (1, 1, 0, 1)
(0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) -> (1, 1, 0, 0)
(0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) -> (1, 0, 1, 1)
(0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0) -> (1, 0, 1, 0)
(0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

3.3 Написать функцию формирования порождающей и проверочной
матриц расширенного кода Хэмминга (𝟐
𝒓
, 𝟐
𝒓 − 𝒓 − 𝟏, 𝟑) на основе
параметра 𝒓, а также таблицы синдромов для всех однократных
ошибок.

In [25]:
def generate_extended_hamming_code(r, N):
    n = 2 ** r - 1

    # Формирование проверочной матрицы H
    H = [[0 for _ in range(r + 1)] for _ in range(N)]
    ii = 0
    for i in range(n):
        x = bin(n - i)[2:]
        if x.count('1') != 1:
            if len(x) < r:
                x = '0' * (r - len(x)) + x
            for j in range(r):
                H[ii][j] = int(x[j])
            H[ii][r] = 1
            ii += 1
    for i in range(r):
        H[ii + i][i] = 1
        H[ii + i][r] = 1
    H[2 ** r - 1][r] = 1
    H = np.array(H)

    # Формирование порождающей матрицы G
    G = []
    for i in range(ii):
        a = [0 for _ in range(ii)]
        a[i] = 1
        G.append(a)
    for i in range(ii):
        for j in range(r):
            G[i].append(H[i][j])
        if sum(G[i]) % 2 != 0:
            G[i].append(1)
        else:
            G[i].append(0)
    G = np.array(G)
    return G, H

3.4. Провести исследование расширенного кода Хэмминга для одно-,
двух-, трёх- и четырёхкратных ошибок для 𝒓 = 𝟐, 𝟑, 𝟒.

In [27]:
print("===== Расширенный код Хэмминга =====", "\n")
G, H = generate_extended_hamming_code(2, 2 ** 2)

study_for_r(2, 2 ** 2, 1)
study_for_r(3, 2 ** 3, 1)
study_for_r(4, 2 ** 4, 1)

===== Расширенный код Хэмминга ===== 

===== Исследование для r = 2 =====
G = 
 [[1 1 1 1]]
H = 
 [[1 1 1]
 [1 0 1]
 [0 1 1]
 [0 0 1]]

 error_table
(1, 0, 0, 0) -> (1, 1, 1)
(0, 1, 0, 0) -> (1, 0, 1)
(0, 0, 1, 0) -> (0, 1, 1)
(0, 0, 0, 1) -> (0, 0, 1)
Внесение 1 ошибок в слово
word =  [1]
code_word =  [1 1 1 1]
error_position = [1]
code_word_with_error =  [1 0 1 1]
syndrome =  [1 0 1]
Ошибка успешно обнаружена и исправлена
code_word =  [1 1 1 1]
code_word_without_error =  [1 1 1 1] 


 error_table
(1, 0, 0, 0) -> (1, 1, 1)
(0, 1, 0, 0) -> (1, 0, 1)
(0, 0, 1, 0) -> (0, 1, 1)
(0, 0, 0, 1) -> (0, 0, 1)
Внесение 2 ошибок в слово
word =  [0]
code_word =  [0 0 0 0]
error_position = [1 2]
code_word_with_error =  [0 1 1 0]
syndrome =  [1 1 0]
Ошибка не была обнаружена и/или не была успешно исправлена
code_word =  [0 0 0 0]
code_word_without_error =  [0 1 1 0]

 error_table
(1, 0, 0, 0) -> (1, 1, 1)
(0, 1, 0, 0) -> (1, 0, 1)
(0, 0, 1, 0) -> (0, 1, 1)
(0, 0, 0, 1) -> (0, 0, 1)
Внесение 3 ошибок